In [1]:
import pandas as pd
excel_filename = 'UterineCancerData.xlsx'
df = pd.read_excel(excel_filename)


# Delete rows with duplicate values in 'col1'
df = df.drop_duplicates(subset=['Sample ID'], keep='last')
df.to_excel(excel_filename, index=False)


In [4]:
# import pandas as pd

# excel_filename = 'UterineCancerData.xlsx'
# df = pd.read_excel(excel_filename)

# # List of columns you want to drop
# columns_to_drop = ['Sample ID','Other Patient ID', 'Form completion date', 'Tissue Source Site', '"International Classification of Diseases for Oncology, Third Edition ICD-O-3 Histology Code"', '"International Classification of Diseases for Oncology, Third Edition ICD-O-3 Site Code"', 'American Joint Committee on Cancer Publication Version Type','ICD-O-3 Histology Code','ICD-O-3 Site Code','AJCC Publication Version Type']

# # Drop the columns
# df = df.drop(columns=columns_to_drop)

# columns_to_update = ['Ethnicity Category', 'Race Category']

for column in columns_to_update:
    df[column] = df[column].fillna('Not-Specified')

# Define the value map for 'Yes' and 'No'
yes_no_value_map = {'Yes': 1, 'No': 0}

# List of columns to apply the Yes/No mapping
columns_with_yes_no = ['In PanCan Pathway Analysis', 'Informed consent verified', 'Neoadjuvant Therapy Type Administered Prior To Resection Text', 
                       'New Neoplasm Event Post Initial Therapy Indicator', 'Prior Diagnosis', 'Radiation Therapy', 
                       'Tissue Prospective Collection Indicator', 'Tissue Retrospective Collection Indicator']

for col in df.columns:
    if col in ['Patient ID']:
        continue

    if df[col].dtype == 'object':
        # Apply Yes/No mapping to specified columns
        if col in columns_with_yes_no:
            df[col] = df[col].astype(str).str.strip()  # Convert to string and strip whitespace
            original_unique_values = set(df[col].dropna().unique())  # Get original unique values before mapping
            df[col] = df[col].map(yes_no_value_map)  # Apply the Yes/No mapping

            # Check and print the mappings applied
            print(f"Column '{col}' mapping:")
            for original_value in original_unique_values:
                if original_value in yes_no_value_map:
                    print(f"'{original_value}' -> {yes_no_value_map[original_value]}")
            print()
        
        elif df[col].apply(lambda x: isinstance(x, str) and ':' in x).any():
            value_map = {value.split(':')[0]: int(value.split(':')[0]) for value in df[col].unique() if isinstance(value, str)}  # Assign values based on text before colon
            df[col] = df[col].map(lambda x: int(x.split(':')[0]) if isinstance(x, str) else x)  # Replace values with integers
            print(f"Column '{col}' mapping:")
            for key, value in value_map.items():
                print(f"'{key}' -> {value}")
            print()

        else:
            unique_values = df[col].dropna().unique()  # Get unique values in the column
            value_map = {value: idx for idx, value in enumerate(unique_values)}  # Assign integers starting from 2
            df[col] = df[col].map(value_map)  # Replace string values with integers
            print(f"Column '{col}' mapping:")
            for key, value in value_map.items():
                print(f"'{key}' -> {value}")
            print()

df.to_excel(excel_filename, index=False)

Column 'Cancer Type Detailed' mapping:
'Uterine Endometrioid Carcinoma' -> 0
'Uterine Serous Carcinoma/Uterine Papillary Serous Carcinoma' -> 1
'Uterine Mixed Endometrial Carcinoma' -> 2

Column 'Overall Survival Status' mapping:
'0' -> 0
'1' -> 1

Column 'Disease Free Status' mapping:
'0' -> 0
'1' -> 1

Column 'Disease-specific Survival status' mapping:
'0' -> 0
'1' -> 1

Column 'Progression Status' mapping:
'1' -> 1
'0' -> 0

Column 'Ethnicity Category' mapping:
'Not Hispanic Or Latino' -> 0
'Not-Specified' -> 1
'Hispanic Or Latino' -> 2

Column 'Race Category' mapping:
'Black or African American' -> 0
'White' -> 1
'Asian' -> 2
'Not-Specified' -> 3
'Native Hawaiian or Other Pacific Islander' -> 4
'American Indian or Alaska Native' -> 5

Column 'Subtype' mapping:
'UCEC_CN_HIGH' -> 0
'UCEC_CN_LOW' -> 1
'UCEC_POLE' -> 2
'UCEC_MSI' -> 3

Column 'Tumor Type' mapping:
'Endometrioid Endometrial Adenocarcinoma' -> 0
'Serous Endometrial Adenocarcinoma' -> 1
'Mixed Serous and Endometrioid Carc

In [5]:
print(df)

       Patient ID  Cancer Type Detailed  Overall Survival Status  \
0    TCGA-2E-A9G8                     0                        0   
1    TCGA-4E-A92E                     0                        0   
2    TCGA-5B-A90C                     0                        0   
3    TCGA-5S-A9Q8                     0                        0   
4    TCGA-A5-A0G1                     1                        1   
..            ...                   ...                      ...   
524  TCGA-QS-A8F1                     1                        1   
525  TCGA-SJ-A6ZI                     0                        0   
526  TCGA-SJ-A6ZJ                     0                        0   
527  TCGA-SL-A6J9                     0                        0   
528  TCGA-SL-A6JA                     0                        0   

     Disease Free Status  Disease-specific Survival status  \
0                    NaN                               0.0   
1                    0.0                               0.0 

In [3]:
import pandas as pd

excel_filename = 'UterineCancerData.xlsx'
df = pd.read_excel(excel_filename)

columns_with_nan = df.columns[df.isna().any()].tolist()

print(columns_with_nan)

['Disease Free Status', 'Disease-specific Survival status', 'Mutation Count', 'Fraction Genome Altered', 'Diagnosis Age', 'MSI MANTIS Score', 'MSIsensor Score', 'Subtype', 'Birth from Initial Pathologic Diagnosis Date', 'Buffa Hypoxia Score', 'Disease Free (Months)', 'Genetic Ancestry Label', 'Last Alive Less Initial Pathologic Diagnosis Date Calculated Day Value', 'Last Communication Contact from Initial Pathologic Diagnosis Date', 'Months of disease-specific survival', 'New Neoplasm Event Post Initial Therapy Indicator', 'Overall Survival (Months)', 'Patient Weight', 'Person Neoplasm Cancer Status', 'Progress Free Survival (Months)', 'Radiation Therapy', 'Ragnum Hypoxia Score', 'Winter Hypoxia Score', 'Aneuploidy Score', 'Tissue Prospective Collection Indicator', 'TMB (nonsynonymous)']


In [4]:
null_counts = df.isnull().sum()

print(null_counts)

Patient ID                                                                  0
Cancer Type Detailed                                                        0
Overall Survival Status                                                     0
Disease Free Status                                                       115
Disease-specific Survival status                                            2
Progression Status                                                          0
Mutation Count                                                             14
Fraction Genome Altered                                                    10
Diagnosis Age                                                               3
MSI MANTIS Score                                                            3
MSIsensor Score                                                             1
Ethnicity Category                                                          0
Race Category                                                   

In [12]:
import pandas as pd

excel_filename = 'UterineCancerData.xlsx'
clinical_data = pd.read_excel(excel_filename)

treatment_filename = 'TreatmentTableUterineCancerData.xlsx'
treatment_data = pd.read_excel(treatment_filename)

# Initialize the treatment column with an empty string
clinical_data['Treatment'] = ''

# Group treatment data by patient_id
grouped_treatment = treatment_data.groupby('Sample ID')['TREATMENT_TYPE'].apply(list)

# Iterate over each patient and construct the treatment sequence
for patient_id, treatments in grouped_treatment.items():
    # Sort treatments to ensure consistent sequence
    treatments.sort()
    # Combine treatments into a single string
    treatment_sequence = '-'.join(treatments)
    # Update the treatment column in clinical_data
    clinical_data.loc[clinical_data['Sample ID'] == patient_id,
'TREATMENT_TYPE'] = treatment_sequence

print(clinical_data)
clinical_data.to_excel(excel_filename, index=False)

       Patient ID        Sample ID  \
0    TCGA-2E-A9G8  TCGA-2E-A9G8-01   
1    TCGA-4E-A92E  TCGA-4E-A92E-01   
2    TCGA-5B-A90C  TCGA-5B-A90C-01   
3    TCGA-5S-A9Q8  TCGA-5S-A9Q8-01   
4    TCGA-A5-A0G1  TCGA-A5-A0G1-01   
..            ...              ...   
524  TCGA-QS-A8F1  TCGA-QS-A8F1-01   
525  TCGA-SJ-A6ZI  TCGA-SJ-A6ZI-01   
526  TCGA-SJ-A6ZJ  TCGA-SJ-A6ZJ-01   
527  TCGA-SL-A6J9  TCGA-SL-A6J9-01   
528  TCGA-SL-A6JA  TCGA-SL-A6JA-01   

                                  Cancer Type Detailed  \
0                       Uterine Endometrioid Carcinoma   
1                       Uterine Endometrioid Carcinoma   
2                       Uterine Endometrioid Carcinoma   
3                       Uterine Endometrioid Carcinoma   
4    Uterine Serous Carcinoma/Uterine Papillary Ser...   
..                                                 ...   
524  Uterine Serous Carcinoma/Uterine Papillary Ser...   
525                     Uterine Endometrioid Carcinoma   
526                  

In [5]:
import pandas as pd

# Load the treatment data
treatment_data_df = pd.read_excel('TreatmentTableUterineCancerData.xlsx')

# Remove the '-01' at the end of the Sample ID and rename the column to Patient ID
treatment_data_df['Sample ID'] = treatment_data_df['Sample ID'].str.replace(r'-01$', '', regex=True)
treatment_data_df.rename(columns={'Sample ID': 'Patient ID'}, inplace=True)

# Save the modified DataFrame back to an Excel file
treatment_data_df.to_excel('TreatmentTableUterineCancerData.xlsx', index=False)


KeyError: 'Sample ID'

In [6]:
import pandas as pd

clinical_data_df = pd.read_excel('UterineCancerData.xlsx')

treatment_data_df = pd.read_excel('TreatmentTableUterineCancerData.xlsx')

treatment_per_patient = treatment_data_df.groupby('Patient ID')['TREATMENT_TYPE'].agg('&'.join).reset_index()

treatment_counts = treatment_data_df.groupby(['Patient ID', 'TREATMENT_TYPE']).size().unstack(fill_value=0).reset_index()

clinical_data_df = clinical_data_df.merge(treatment_per_patient, on='Patient ID', how='left').fillna('')
clinical_data_df = clinical_data_df.merge(treatment_counts, on='Patient ID', how='left').fillna(0)


clinical_data_df = clinical_data_df.rename(columns={'TREATMENT_TYPE': 'treatment'})

clinical_data_df.to_excel('UterineCancerData.xlsx', index=False)